In [2]:
import requests
import json
import re

# Global Variables 
### ( FOR STATIC USE )

In [107]:
correct_adrs=[
    "2nd Phase, 374/B, 80 Feet Rd, Mysore Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560050",
    "2nd Phase, 374/B, 80 Feet Rd, Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560050",
    "374/B, 80 Feet Rd, State Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bangalore. 560050",
]
incorrect_adrs=[
    "2nd Phase, 374/B, 80 Feet Rd, Mysore Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560095",
    "Colony, Bengaluru, Karnataka 560050"
]
states=["Andaman & Nicobar","Andhra Pradesh","Arunachal Pradesh","Assam","Bihar","Chandigarh","Chattisgarh","Dadra & Nagar Haveli","Daman & Diu","Delhi","Goa","Gujarat","Haryana","Himachal Pradesh","Jammu & Kashmir","Jharkhand","Karnataka","Kerala","Lakshadweep","Madhya Pradesh","Maharashtra","Manipur","Meghalaya","Mizoram","Nagaland","Odisha","Pondicherry","Punjab","Rajasthan","Sikkim","Tamil Nadu","Telangana","Tripura","Uttar Pradesh","Uttarakhand","West Bengal"]
res_keys=["Description","BranchType","DeliveryStatus","Circle","District","Division","Region","Block","State","Country","Pincode"]

# Utility Functions

In [133]:
url_by_pin = lambda pincode: f"https://api.postalpincode.in/pincode/{pincode}"
url_by_city = lambda city_name: f"https://api.postalpincode.in/postoffice/{city_name}"
get_pincode = lambda text : re.findall(r'\b\d{6}\b', text)
substr_in_str = lambda sub_str,string: True if string.lower().find(sub_str.lower())!=-1 else False

def print_json(payload):
    print(json.dumps(payload,indent=2))

# Extracts city from address
def get_city(address):
    lis=[s.strip() for s in address.split(',')]
    last_item=lis[-1]
    has_state_name=False
    for state in states:
        if(substr_in_str(state,last_item)):
            has_state_name=True
            break
    city = lis[-3] if has_state_name else lis[-2]
    return city

# Extracts street name from address
"""
Works on the logic:
Splits based on commas and remove white spaces
Returns the first index where all characters are alphabet
"""
def get_street_from_address(address):
    pattern = "^[a-zA-Z\s]+$"
    lis=[s.strip() for s in address.split(',')]
    for item in lis:
        if re.match(pattern, item):
            return item

# Api Based Fuctions

In [66]:
# FUNCTION TO RETURN POST OFFICES BY PINCODE 
def offices_by_pin(pin):
    res=requests.get(url_by_pin(pin))
    res=res.json()[0]
    # print_json(res)
    if res.get("Status")!='Success':
        print_json(res)
        return []
    return res.get("PostOffice")

# FUNCTION TO RETURN POST OFFICES BY NAME 
def offices_by_city(name):
    res=requests.get(url_by_city(name))
    res=res.json()[0]
    # print_json(res)
    if res.get("Status")!='Success':
        print_json(res)
        return None
    return res.get("PostOffice")

In [127]:
# print(offices_by_pin(560050))
# offices_by_city('Srinivasa Nagar')

# Api Response Structure 
### (Single Array Element)

In [126]:
 {'Name': 'Srinivasa Nagar Colony',
  'Description': None,
  'BranchType': 'Branch Post Office',
  'DeliveryStatus': 'Delivery',
  'Circle': 'Andhra Pradesh',
  'District': 'Krishna',
  'Division': 'Vijayawada',
  'Region': 'Vijayawada',
  'State': 'Andhra Pradesh',
  'Country': 'India',
  'Pincode': '521175'}

{'Name': 'Srinivasa Nagar Colony',
 'Description': None,
 'BranchType': 'Branch Post Office',
 'DeliveryStatus': 'Delivery',
 'Circle': 'Andhra Pradesh',
 'District': 'Krishna',
 'Division': 'Vijayawada',
 'Region': 'Vijayawada',
 'State': 'Andhra Pradesh',
 'Country': 'India',
 'Pincode': '521175'}

In [141]:
def is_address_valid(address):
    address=address.replace('Bengaluru','Bangalore')
    pincode=get_pincode(address)[0]
    
    city=get_city(address)
    city_verified=map_calculated_city_and_pincode(city,pincode)
    
#     offices=offices_by_pin(pincode)
#     for office in offices:
#         res=match_pin(address,office)
    print(address)
    print("city name:" + city)
    if(not city_verified):
        print('------------Address is Invalid-------------------')
        return False
    print('------------Address is Valid-------------')
    return True

In [128]:
# Checks the pincode from address is presnet in the list of post-offices that we get from API by passing city-name 
def map_calculated_city_and_pincode(cityname,pincode):
    offices=offices_by_city(cityname)
    if(offices):
        for office in offices:
            if(int(office['Pincode'])==pincode):
                return True
    return False

In [139]:
# INCOMPETE
# Points based algorithm to match keys in the address and post-offices recieved by pincode
def match_pin(address,office):
    state = substr_in_str(address,office['State'])
    district = substr_in_str(address,office['District'])
    circle = substr_in_str(address,office['Circle'])
    print({"state":state,"district":district,"circle":circle})
    print_json(office)
    

# Notes
<ol>
    <li>The algorithym's incomplete and heavily prone to erros</li>
    <li>
        Current approach:
        <br>
        <ul>
            <li>We exract the pincode and city name from address</li>
            <li>Then get the list of post offices in that city</li>
            <li>We check if the pincode is present in the list of offices that we get from api</li>
            <li>The address is marked valid if the above condition is met and invalid in the opposite case</li>
        </ul>
    </li>
    <li>
        Initial approach was to work out a point based system where:
        <br>
        <ul>
            <li>We exract the pincode from address</li>
            <li>Then get the list of post offices in that pincode</li>
            <li>Then iterate through thoes offices</li>
            <li>Each parameter of the office object would have some particular weightage</li>
            <li>And based on score we can calculate if the address is valid or not and the accuracy percentage</li>
        </ul>
    </li>
</ol>

# Address Requirements

<li>Address third or second last index should be the city name</li>
<li>Address must have pincode</li>

# Primary Input

In [142]:
address=incorrect_adrs[0]

is_address_valid(address)

2nd Phase, 374/B, 80 Feet Rd, Mysore Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bangalore, Karnataka 560095
city name:Srinivasa Nagar
------------Address is Invalid-------------------


False